# NLTK TOKENIZER AND TF-IDF VECTORIZER

In [1]:
import sys
sys.path.append('../')
from tokenizer import tokenizer
from vectorizer import vectorizer

import numpy as np
import pandas as pd
import sklearn as sk

[nltk_data] Downloading package stopwords to /home/jean-
[nltk_data]     michel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv('../../_data/Reviews.csv') # Loading the dataset
X, y = data['Text'], data['Score']

In [3]:
X, y = X[:], y[:]

In [4]:
#tokenized_documents = tokenizer(X.copy())

In [5]:
#X, vect = vectorizer(tokenized_documents)

In [6]:
batch_size = 10000

print(f"Processing data of length {len(X)} in batches of size {batch_size}")
X_vect_batches = []
for start in range(0, len(X), batch_size):
    end = start + batch_size
    print(f"Processing batch from index {start} to {end}")
    batch_X = X[start:end].copy()
    tokenized_documents = tokenizer(batch_X)
    X_vect, _ = vectorizer(tokenized_documents)
    # Store the processed batch
    X_vect_batches.append(X_vect)

X = np.vstack(X_vect_batches)


Processing data of length 568454 in batches of size 10000
Processing batch from index 0 to 10000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 10000 to 20000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 20000 to 30000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 30000 to 40000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 40000 to 50000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 50000 to 60000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 60000 to 70000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 70000 to 80000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 80000 to 90000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 90000 to 100000
Number of tokens:  0
Number of sentences:  0
Processing batch from index 100000 to 110000
Number of tokens:  0

In [7]:
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

print(X_train[0])

(454763, 300) (113691, 300) (454763,) (113691,)
[0.69675133 0.46625919 0.19659357 0.54161421 0.49475396 0.68864423
 0.77237797 0.61296189 0.64803631 0.67720143 0.53871329 0.43821322
 0.57999926 0.60784278 0.64792152 0.49237756 0.62410983 0.48551687
 0.66650609 0.6353156  0.28711372 0.57419198 0.26327911 0.58957649
 0.58278467 0.46489851 0.33019595 0.34122713 0.50042139 0.47023063
 0.23547583 0.40088003 0.36770556 0.21215492 0.44290871 0.29329761
 0.39150614 0.41075528 0.67269826 0.2800273  0.59147843 0.74466023
 0.64935439 0.45447142 0.53504712 0.59418841 0.35070694 0.20508571
 0.42417751 0.48023969 0.40409482 0.63481863 0.44575853 0.45432337
 0.44045728 0.50378316 0.26800294 0.76815876 0.40490527 0.42683939
 0.6215097  0.43453887 0.51992281 0.53506951 0.58080606 0.52360652
 0.54309204 0.47171951 0.43835253 0.65036521 0.4512514  0.70861387
 0.63950446 0.58614768 0.42354527 0.74754527 0.66673732 0.70711648
 0.60580582 0.34354116 0.45962957 0.76664349 0.34378702 0.61320123
 0.4956486  0.

# Feed Forward Neural Network
### Model starts here

In [9]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.layers import LeakyReLU
from sklearn import metrics
import datetime
import os

2024-05-11 00:24:11.201302: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 00:24:11.294821: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-11 00:24:11.647464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 00:24:12.680625: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True),
    ModelCheckpoint('model_best.keras', save_best_only=True, monitor='val_loss', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=0.00001)
]

In [16]:
def batch_generator(X, y, batch_size, num_classes):
    num_samples = X.shape[0]
    while True:
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            X_batch = X[start:end]
            y_batch = to_categorical(y[start:end], num_classes=num_classes)
            yield X_batch, y_batch

# Setup the model
inputs = Input(shape=(X_train.shape[1],))
x = Dense(256)(inputs)
x = LeakyReLU()(x)
x = Dropout(0.1)(x)
x = Dense(128)(x)
x = LeakyReLU()(x)
residual = Dense(64)(x)
x = LeakyReLU()(residual)

x = Dense(64)(x)
x = LeakyReLU()(x)
x = Dropout(0.1)(x)
x = Add()([x, residual])
x = Dense(32)(x)
x = LeakyReLU()(x)
x = Dense(16)(x)
x = LeakyReLU()(x)

outputs = Dense(np.max(y_train) + 1, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])

# Define the generator
train_generator = batch_generator(X_train, y_train, 32, np.max(y_train) + 1)


In [17]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=int(np.ceil(X_train.shape[0] / 32)),
    epochs=100,
    callbacks=callbacks,
)

Epoch 1/100


14212/14212 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.6543 - loss: 0.9964 - learning_rate: 1.0000e-04
Epoch 2/100
  153/14212 ━━━━━━━━━━━━━━━━━━━━ 14s 997us/step - accuracy: 0.6698 - loss: 0.9077

/home/jean-michel/Epita/nlp/.venv/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/jean-michel/Epita/nlp/.venv/lib/python3.10/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/home/jean-michel/Epita/nlp/.venv/lib/python3.10/site-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


14212/14212 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.6719 - loss: 0.9023 - learning_rate: 1.0000e-04
Epoch 3/100
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.6757 - loss: 0.8897 - learning_rate: 1.0000e-04
Epoch 4/100
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.6777 - loss: 0.8811 - learning_rate: 1.0000e-04
Epoch 5/100
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.6802 - loss: 0.8743 - learning_rate: 1.0000e-04
Epoch 6/100
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.6814 - loss: 0.8691 - learning_rate: 1.0000e-04
Epoch 7/100
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.6826 - loss: 0.8640 - learning_rate: 1.0000e-04
Epoch 8/100
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.6844 - loss: 0.8594 - learning_rate: 1.0000e-04
Epoch 9/100
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.6854 - loss: 0.8549 - learning_rate: 1.0000e-04
Epoch 10/100
14212/14212 ━━━━━━━━━━━━━━━━━━━━ 15s 1m

In [18]:
test_generator = batch_generator(X_test, y_test, batch_size=32, num_classes=np.max(y_train) + 1)

In [19]:
# Evaluate the model
predictions_prob = model.predict(test_generator, steps=int(np.ceil(X_test.shape[0] / 32)))
predictions = np.argmax(predictions_prob, axis=1)

# Metrics
print("Confusion Matrix:")
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

print("\nClassification Report:")
print(metrics.classification_report(y_test, predictions))

3553/3553 ━━━━━━━━━━━━━━━━━━━━ 3s 788us/step
Confusion Matrix:
[[ 5982   525   228   193  3398]
 [ 1322   924   463   430  2716]
 [  903   503  1115  1199  4765]
 [  456   207   489  2591 12380]
 [  886   183   311  1186 70336]]

Classification Report:
              precision    recall  f1-score   support

           1       0.63      0.58      0.60     10326
           2       0.39      0.16      0.23      5855
           3       0.43      0.13      0.20      8485
           4       0.46      0.16      0.24     16123
           5       0.75      0.96      0.84     72902

    accuracy                           0.71    113691
   macro avg       0.53      0.40      0.42    113691
weighted avg       0.66      0.71      0.66    113691



In [15]:
# # Save the model with current date and time in model folder

# Create a folder named _models in the current directory
if not os.path.exists('_models'):
    os.makedirs('_models')
    
model.save(f'_models/config3_feedforward_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')